In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
import glob
import pandas as pd
import os
import seaborn as sns

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
from pathlib import Path
import scipy.stats
import time

from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_plotting_utils import *
from open_spiel.python.examples.ubc_sample_game_tree import sample_game_tree, flatten_trees, flatten_tree
from open_spiel.python.examples.ubc_clusters import projectPCA, fitGMM
from open_spiel.python.examples.ubc_utils import *
import open_spiel.python.examples.ubc_dispatch as dispatch

from auctions.webutils import *

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


from open_spiel.python.examples.ubc_cma import *

output_notebook()
from open_spiel.python.games.clock_auction_base import InformationPolicy, ActivityPolicy, UndersellPolicy
from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_decorators import TakeSingleActionDecorator, TremblingAgentDecorator, ModalAgentDecorator
from open_spiel.python.algorithms.outcome_sampling_mccfr import OutcomeSamplingSolver
from open_spiel.python.algorithms.external_sampling_mccfr import ExternalSamplingSolver
from open_spiel.python.algorithms.exploitability import nash_conv


Loading BokehJS ...

In [3]:
# game = pyspiel.load_game("python_clock_auction", dict(filename='parameters.json'))
num_iterations = 999_999_000

In [29]:
def run_solver(modal=True, time_limit_seconds=60, seed=1234, regret_matching_plus=False, linear_averaging=False):
    np.random.seed(seed)
    # game = pyspiel.load_game("python_clock_auction", dict(filename='/apps/open_spiel/configs/jun24/jun24_1_high_speed.json')) # Reload the game object so caching isn't a part of this
    game = pyspiel.load_game("python_clock_auction", dict(filename='/apps/open_spiel/configs/jun24/jun24_1_base.json')) # Reload the game object so caching isn't a part of this
    # game = pyspiel.load_game("python_clock_auction", dict(filename='parameters.json'))

    solver = OutcomeSamplingSolver(game, regret_matching_plus=regret_matching_plus, linear_averaging=linear_averaging)
    start = time.time()
    
    for i in tqdm(range(num_iterations)):
        solver.iteration()
        if time.time() - start > time_limit_seconds:
            print(f"Aborting after {i} iterations")
            break
    policy = solver.average_policy()
    
    if modal:
        env_and_policy = make_env_and_policy(game, dict(solver_type='cfr'))
        for agent in env_and_policy.agents:
            agent.policy = policy
        for i in range(len(env_and_policy.agents)):
            env_and_policy.agents[i] = ModalAgentDecorator(env_and_policy.agents[i])
        policy = env_and_policy.make_policy(string_only=True)
        
    return nash_conv(game, policy)


In [31]:
run_solver(modal=True)

  0%|                                                                                                                                                                                                                         | 70481/999999000 [00:59<236:27:08, 1174.69it/s]


Aborting after 70481 iterations


8.449999999999996

In [30]:
run_solver(linear_averaging=True)

  0%|                                                                                                                                                                                                                          | 57065/999999000 [00:59<292:02:47, 951.09it/s]


Aborting after 57065 iterations


19.595000000000013

In [32]:
run_solver(regret_matching_plus=True)

  0%|                                                                                                                                                                                                                          | 42196/999999000 [01:00<394:57:57, 703.27it/s]


Aborting after 42196 iterations


29.69850000000001

In [33]:
run_solver(regret_matching_plus=True, linear_averaging=True)

  0%|                                                                                                                                                                                                                          | 42684/999999000 [00:59<390:26:51, 711.40it/s]


Aborting after 42684 iterations


29.360749999999996

In [ ]:
game = pyspiel.load_game("python_clock_auction", dict(filename='/apps/open_spiel/configs/jun24/jun24_1_high_speed.json')) # Reload the game object so caching isn't a part of this
solver = OutcomeSamplingSolver(game)
for i in tqdm(range(num_iterations)):
    solver.iteration()

In [ ]:
policy = solver.average_policy()


In [ ]:


# nash_conv(game, policy)

In [ ]:
env_and_policy = make_env_and_policy(game, dict())
for agent in env_and_policy.agents:
    agent.policy = policy
nash_conv(game, env_and_policy.make_policy(string_only=True))